### Flant5 Train
Training file to fine-tune the flan-t5 base model. Flan-T5 is a variant of the T5 (Text-To-Text Transfer Transformer) model that has been fine-tuned using the FLAN (Fine-tuned Language Net) methodology.

#### Step 1: Install Required Dependencies

In [1]:
!pip install evaluate
!pip install sacrebleu
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 825.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 901.3 kB/s eta 0:00:00--:--


Load the datasets, Large Language Model (LLM) and tokenizer.

In [2]:
import os
import torch
import numpy as np
from datasets import load_dataset
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import bert_score

#### Step 2: Preprocess dataset
The restructure_json function processes a list of JSON file names, reads each file, extracts specific fields ('disfluent' and 'original'), and writes the restructured data to new output files. It constructs file paths dynamically and uses JSON operations to read and write the data.

In [3]:
def restructure_json(file_names):
    """
    Restructures the JSON files specified by the given list of file names.

    Parameters:
    file_names (list): A list of file names (without extension) to be processed.

    Returns:
    None
    """
    for file_name in file_names:
        input_path = os.path.join(os.getcwd(), f"{file_name}.json")
        output_path = os.path.join(os.getcwd(), f"{file_name}_output.json")

        #print(input_path)
        #print(output_path)

        with open(input_path, 'r') as f:
            raw_data = json.load(f)
        #print(raw_data)

        dataset = [{'disfluent': item['disfluent'], 'original': item['original']} for item in raw_data.values()]

        with open(output_path, 'w') as f:
            json.dump(dataset, f, indent=4)

In [4]:
# Mention the train, dev and test file names without extension and if using holdout datasets, rename the holdout dataset to test dataset
# Please make sure that file name defined below should have .json extension
file_names = ["train", "dev", "test"]
restructure_json(file_names)

Load the preprocessed train, dev and test dataset.

In [5]:
data_files = {k: os.path.join(os.getcwd(), f"{k}_output.json") for k in ["train", "dev", "test"]}
dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Define your model and tokenizer. Make sure to use the correct model name and tokenizer name

In [6]:
# Initialize the model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name, return_tensors="pt")
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

cuda


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

#### Step 3: Tokenize the input and target
Define the function to tokenize disfluent and original questions representing inputs and targets with maximum length 

In [7]:
max_length = 512
def tokenize_function(examples):
    """
    Preprocesses the input examples for training a disfluency detection model.

    Args:
        examples (dict): A dictionary containing the input examples with 'disfluent' and 'original' keys.

    Returns:
        dict: A dictionary containing the preprocessed model inputs with 'input_ids', 'attention_mask', and 'labels' keys.
    """

    inputs = examples['disfluent']
    targets = examples['original']
    model_inputs = tokenizer(inputs, max_length = max_length, truncation = True, padding="max_length")
    labels = tokenizer(targets, max_length = max_length, truncation = True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset.map(tokenize_function, batched=True, remove_columns=['disfluent', 'original'])

Map:   0%|          | 0/7182 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

In [8]:
# Data collator to handle dynamic padding and other pre-processing requirements
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### Step 4: Define the metric
Metric function to evaluate the model. Model is evaluated on sacrebleu and Bert F1 score.

In [9]:
# Load BLEU metrics
sacrebleu = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    """
    Compute evaluation metrics for prediction results.
    Args:
        eval_pred (tuple): A tuple containing predictions and labels.
    Returns:
        dict: A dictionary containing the computed evaluation metrics.
            - "bleu" (float): The BLEU score.
            - "Bert Score F1" (str): The average BERTScore F1 score.
    """
    predictions, labels = eval_pred
    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]  # SacreBLEU expects a list of references for each prediction

    sacrebleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate BERTScore
    P, R, F1 = bert_score.score(decoded_preds, decoded_labels, lang="en", verbose=True)

    return {
        "bleu": sacrebleu_result["score"],
        "Bert Score F1": f"{F1.mean().item():.4f}"
    }

#### Step 5: Define the training arguments

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/output/",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True,
    learning_rate=0.00017105019776419224,  # optimum learning rate found by hyperparameter tuning (using optuna)
    weight_decay=0.05238804154208795 # optimum weight decay found by hyperparameter tuning (using optuna)
)

#### Step 6: Intialize the trainer

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['dev'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


#### Step 7: Train the model

In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Bert score f1
1,0.831800,0.004827,89.607818,0.9888
2,0.004400,0.004769,89.818564,0.9893
3,0.003100,0.005379,90.199489,0.9900
4,0.002100,0.006171,90.297948,0.9899
5,0.001800,0.007096,90.346927,0.9899


...

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 3.65 seconds, 274.25 sentences/sec


Trainer is attempting to log a value of "0.9899" of type <class 'str'> for key "eval/Bert Score F1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=4490, training_loss=0.09538775702361806, metrics={'train_runtime': 7869.6704, 'train_samples_per_second': 4.563, 'train_steps_per_second': 0.571, 'total_flos': 2.458963652640768e+16, 'train_loss': 0.09538775702361806, 'epoch': 5.0})

In [ ]:
# Save the model and tokenizer
# model.save_pretrained("/kaggle/working/disfl-fine-tuned-FlanT5-model")
# tokenizer.save_pretrained("/kaggle/working/disfl-fine-tuned-FlanT5-tokenizer")